# Install the required libraries

In [2]:
!pip3 install torch torchvision

     |███████████▊                    | 326.1 MB 115.6 MB/s eta 0:00:05  |█                               | 27.1 MB 8.1 MB/s eta 0:01:48

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |███████████████████████▍        | 649.7 MB 136.3 MB/s eta 0:00:02

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 890.2 MB 8.6 kB/s  eta 0:00:011
     |████████████████████████████████| 24.3 MB 80.7 MB/s eta 0:00:01
     |████████████████████████████████| 21.0 MB 71.1 MB/s eta 0:00:01
     |███████████████▏                | 263.9 MB 133.3 MB/s eta 0:00:03     |████████████▉                   | 222.6 MB 130.0 MB/s eta 0:00:03

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 557.1 MB 8.5 kB/s  eta 0:00:011
     |███████████████████████████▉    | 275.7 MB 138.4 MB/s eta 0:00:01

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 317.1 MB 30 kB/s 
     |████████████████████████████████| 849 kB 69.6 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 3.5 MB/s s eta 0:00:01
     |████████████████████████████████| 3.3 MB 91.8 MB/s eta 0:00:01
     |████████████████████████████████| 17.1 MB 52.1 MB/s eta 0:00:01
     |████████████████████████████████| 140 kB 107.6 MB/s eta 0:00:01
     |████████████████████████████████| 161 kB 112.5 MB/s eta 0:00:01


In [4]:
!pip3 install matplotlib

     |████████████████████████████████| 11.8 MB 6.7 MB/s eta 0:00:01
     |████████████████████████████████| 296 kB 108.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 91.9 MB/s eta 0:00:01
     |████████████████████████████████| 965 kB 90.2 MB/s eta 0:00:01


In [6]:
!pip3 install pandas

     |████████████████████████████████| 12.2 MB 8.0 MB/s eta 0:00:01
     |████████████████████████████████| 498 kB 104.4 MB/s eta 0:00:01


In [19]:
!pip3 install torch-summary

# Import the required libraries

In [1]:
import torch
torch.cuda.empty_cache()
import math
import torchvision
import torchvision.transforms as transforms
import random
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
from torch.utils.data import DataLoader,random_split
import torch.nn.functional as F
from collections import OrderedDict
import csv
import pandas as pd
import numpy as np
from torchsummary import summary
import torch.optim as optim
import os
from collections import defaultdict
from torch.optim.optimizer import Optimizer


# Utility for Cutout Transformation
Ref : https://github.com/uoguelph-mlrg/Cutout

In [2]:
class Cutout(object):
    """Randomly mask out one or more patches from an image.

    Args:
        n_holes (int): Number of patches to cut out of each image.
        length (int): The length (in pixels) of each square patch.
    """
    def __init__(self, n_holes, length):
        self.n_holes = n_holes
        self.length = length

    def __call__(self, img):
        """
        Args:
            img (Tensor): Tensor image of size (C, H, W).
        Returns:
            Tensor: Image with n_holes of dimension length x length cut out of it.
        """
        h = img.size(1)
        w = img.size(2)

        mask = np.ones((h, w), np.float32)

        for n in range(self.n_holes):
            y = np.random.randint(h)
            x = np.random.randint(w)

            y1 = np.clip(y - self.length // 2, 0, h)
            y2 = np.clip(y + self.length // 2, 0, h)
            x1 = np.clip(x - self.length // 2, 0, w)
            x2 = np.clip(x + self.length // 2, 0, w)

            mask[y1: y2, x1: x2] = 0.

        mask = torch.from_numpy(mask)
        mask = mask.expand_as(img)
        img = img * mask

        return img

# Utility for Lookahead Optimizer 
Ref : https://github.com/michaelrzhang/lookahead

In [3]:

class Lookahead(Optimizer):
    r"""PyTorch implementation of the lookahead wrapper.

    Lookahead Optimizer: https://arxiv.org/abs/1907.08610
    """

    def __init__(self, optimizer, la_steps=5, la_alpha=0.8, pullback_momentum="none"):
        """optimizer: inner optimizer
        la_steps (int): number of lookahead steps
        la_alpha (float): linear interpolation factor. 1.0 recovers the inner optimizer.
        pullback_momentum (str): change to inner optimizer momentum on interpolation update
        """
        self.optimizer = optimizer
        self._la_step = 0  # counter for inner optimizer
        self.la_alpha = la_alpha
        self._total_la_steps = la_steps
        pullback_momentum = pullback_momentum.lower()
        assert pullback_momentum in ["reset", "pullback", "none"]
        self.pullback_momentum = pullback_momentum

        self.state = defaultdict(dict)

        # Cache the current optimizer parameters
        for group in optimizer.param_groups:
            for p in group['params']:
                param_state = self.state[p]
                param_state['cached_params'] = torch.zeros_like(p.data)
                param_state['cached_params'].copy_(p.data)
                if self.pullback_momentum == "pullback":
                    param_state['cached_mom'] = torch.zeros_like(p.data)

    def __getstate__(self):
        return {
            'state': self.state,
            'optimizer': self.optimizer,
            'la_alpha': self.la_alpha,
            '_la_step': self._la_step,
            '_total_la_steps': self._total_la_steps,
            'pullback_momentum': self.pullback_momentum
        }

    def zero_grad(self):
        self.optimizer.zero_grad()

    def get_la_step(self):
        return self._la_step

    def state_dict(self):
        return self.optimizer.state_dict()

    def load_state_dict(self, state_dict):
        self.optimizer.load_state_dict(state_dict)

    def _backup_and_load_cache(self):
        """Useful for performing evaluation on the slow weights (which typically generalize better)
        """
        for group in self.optimizer.param_groups:
            for p in group['params']:
                param_state = self.state[p]
                param_state['backup_params'] = torch.zeros_like(p.data)
                param_state['backup_params'].copy_(p.data)
                p.data.copy_(param_state['cached_params'])

    def _clear_and_load_backup(self):
        for group in self.optimizer.param_groups:
            for p in group['params']:
                param_state = self.state[p]
                p.data.copy_(param_state['backup_params'])
                del param_state['backup_params']

    @property
    def param_groups(self):
        return self.optimizer.param_groups

    def step(self, closure=None):
        """Performs a single Lookahead optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = self.optimizer.step(closure)
        self._la_step += 1

        if self._la_step >= self._total_la_steps:
            self._la_step = 0
            # Lookahead and cache the current optimizer parameters
            for group in self.optimizer.param_groups:
                for p in group['params']:
                    param_state = self.state[p]
                    p.data.mul_(self.la_alpha).add_(param_state['cached_params'], alpha=1.0 - self.la_alpha)  # crucial line
                    param_state['cached_params'].copy_(p.data)
                    if self.pullback_momentum == "pullback":
                        internal_momentum = self.optimizer.state[p]["momentum_buffer"]
                        self.optimizer.state[p]["momentum_buffer"] = internal_momentum.mul_(self.la_alpha).add_(
                            1.0 - self.la_alpha, param_state["cached_mom"])
                        param_state["cached_mom"] = self.optimizer.state[p]["momentum_buffer"]
                    elif self.pullback_momentum == "reset":
                        self.optimizer.state[p]["momentum_buffer"] = torch.zeros_like(p.data)

        return loss

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch


In [5]:
device

'cuda'

# Model Configuration for Resnet-18 
Ref : https://github.com/kuangliu/pytorch-cifar

In [6]:
#Model

class BasicBlock(nn.Module):

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes,
                          kernel_size=kernel_size, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)
  
  
    def _make_layer(self, block, planes, num_blocks, stride):
        downsample = None
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)


    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, pool_size)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


In [7]:
# Finalized Model Configuration :

layers=[1, 1, 1, 1]
kernel_size = 1
pool_size = 4
model = ResNet(BasicBlock, layers)

print(summary(model))

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            1,728
├─BatchNorm2d: 1-2                       128
├─Sequential: 1-3                        --
|    └─BasicBlock: 2-1                   --
|    |    └─Conv2d: 3-1                  36,864
|    |    └─BatchNorm2d: 3-2             128
|    |    └─Conv2d: 3-3                  36,864
|    |    └─BatchNorm2d: 3-4             128
|    |    └─Sequential: 3-5              --
├─Sequential: 1-4                        --
|    └─BasicBlock: 2-2                   --
|    |    └─Conv2d: 3-6                  73,728
|    |    └─BatchNorm2d: 3-7             256
|    |    └─Conv2d: 3-8                  147,456
|    |    └─BatchNorm2d: 3-9             256
|    |    └─Sequential: 3-10             8,448
├─Sequential: 1-5                        --
|    └─BasicBlock: 2-3                   --
|    |    └─Conv2d: 3-11                 294,912
|    |    └─BatchNorm2d: 3-12            512
|    |    └─Conv2d: 3-13             

# Traing and Testing Methods for model 
Ref : https://github.com/kuangliu/pytorch-cifar

Steps involved : 
1. Define the model hyperparameters.
2. Load the CIFAR-10 dataset. 
3. Perform data transformation on train and test data. 
4. Define the scheduler and optimizer. 
5. Train the model for the specified epochs. 
6. Test the model for each epoch
7. Save the model results for each epoch in a json file. 

In [9]:
# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    acc = 100.*correct/total
    loss = 100.*train_loss/total
    print('Train Loss: %.3f | Train Acc: %.3f%% (%d/%d)'
                     % (loss, acc, correct, total))
    model_results[str(epoch)] =  {"train" : {"acc" : acc,"loss" : loss},"test" : {}}
    
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            # print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            #              % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    loss = 100.*test_loss/total
    print('Test Loss: %.3f | Test Acc: %.3f%% (%d/%d)'
                     % (loss, acc, correct, total))
    model_results[str(epoch)]['test']  = {"acc" : acc,"loss" : loss}
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('Adam'):
            os.mkdir('Adam')
        torch.save(state, './Adam/ckpt_512.pth')
        best_acc = acc
        

#Model Parameters

batch_size = 512
lr = 1e-2
optim_param = {'la_steps':5,
               'la_alpha':0.5
              }
resume = False 
model_results = {}

#Load model
        
net = model
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)

if resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('Adam'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('./Adam/ckpt_512.pth')
    net.load_state_dict(checkpoint['net'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    Cutout(n_holes=1, length=8)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=batch_size, shuffle=False, num_workers=2)


criterion = nn.CrossEntropyLoss()
base_optim = optim.Adam(net.parameters(), lr=lr)

Q = math.floor(len(trainset)/batch_size)
optimizer = Lookahead(base_optim, **optim_param)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=Q)

epochs = 50

for epoch in range(start_epoch, start_epoch+epochs+1):
    train(epoch)
    test(epoch)
    scheduler.step()

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
Train Loss: 0.350 | Train Acc: 34.694% (17347/50000)
Test Loss: 0.358 | Test Acc: 39.670% (3967/10000)
Saving..

Epoch: 1
Train Loss: 0.256 | Train Acc: 52.280% (26140/50000)
Test Loss: 0.276 | Test Acc: 51.390% (5139/10000)
Saving..

Epoch: 2
Train Loss: 0.207 | Train Acc: 62.016% (31008/50000)
Test Loss: 0.247 | Test Acc: 58.190% (5819/10000)
Saving..

Epoch: 3
Train Loss: 0.177 | Train Acc: 67.768% (33884/50000)
Test Loss: 0.219 | Test Acc: 63.690% (6369/10000)
Saving..

Epoch: 4
Train Loss: 0.156 | Train Acc: 71.648% (35824/50000)
Test Loss: 0.133 | Test Acc: 76.790% (7679/10000)
Saving..

Epoch: 5
Train Loss: 0.136 | Train Acc: 75.724% (37862/50000)
Test Loss: 0.156 | Test Acc: 75.200% (7520/10000)

Epoch: 6
Train Loss: 0.120 | Train Acc: 78.686% (39343/50000)
Test Loss: 0.148 | Test Acc: 75.520% (7552/10000)

Epoch: 7
Train Loss: 0.109 | Train Acc: 80.484% (40242/50000)
Test

## Save the model result as json

In [10]:
import json
with open('result_adam_512.json', 'w') as fp:
    json.dump(model_results, fp)

## Check the checkpoint model with highest accuracy

In [11]:
checkpoint = torch.load('./Adam/ckpt_512.pth')
net.load_state_dict(checkpoint['net'])
best_acc = checkpoint['acc']

In [12]:
best_acc

91.26